# Resistor Values

How to choose resistor values for summing mixer, to implement panning?

In [1]:
import altair as alt
import numpy as np
import pandas as pd

## Constant Power Panning

Following the formula given on [DSP StackExchange](https://dsp.stackexchange.com/a/21736/72857), based on _C. Roads: The Computer Music Tutorial_:

For an angle $\phi$ (ranging from -45° to 45°, where 0° denotes the center), the amplitude should be scaled by 

$$
    \frac{\sqrt{2}}{2}\left(\cos{\phi} + \sin{\phi}\right), \quad
    \frac{\sqrt{2}}{2}\left(\cos{\phi} - \sin{\phi}\right)
$$

for the left and right channels, respectively.

The extreme values of the angles have one of the amplitudes at 0, ie, all audio coming from one speaker.
An angle of 45° signifies fully left, so the sorting on the charts may be confusing.

In [2]:
def const_power_left(phi):
    return np.sqrt(2)/2 * (np.cos(phi) + np.sin(phi))

def const_power_right(phi):
    return const_power_left(-phi)

In [24]:
angles = np.arange(-45, 45 + 1, 1)

In [44]:
def amps(degrees):
    radians = np.radians(degrees)
    df = pd.DataFrame(dict(
        angle=degrees,
        radian=radians,
        left=const_power_left(radians),
        right=const_power_right(radians)
    ))
    return df

In [45]:
df = amps(angles)
df.head()

,angle,radian,left,right
0,-45,-0.785398,7.850462e-17,1.000000
1,-44,-0.767945,1.745241e-02,0.999848
2,-43,-0.750492,3.489950e-02,0.999391
3,-42,-0.733038,5.233596e-02,0.998630
4,-41,-0.715585,6.975647e-02,0.997564


In [72]:
alt.Chart(df).mark_line(color="blue").encode(x="angle", y="left") \
+ alt.Chart(df).mark_line(color="red").encode(x="angle", y="right")

alt.LayerChart(...)

In [47]:
# all on a circle of radius 1 (== constant power)
np.max(np.abs(df["left"]**2 + df["right"]**2 - 1.0))

np.float64(4.440892098500626e-16)

## Nearness positions

Nearness features 7 panning position, in steps of 22.5° around the center.

In [106]:
step = 22.5  # following the description
# step = 30  # trying a more intuitive value?
step = step / 2  # go from 180° intuition to 90° formula
nearness_angles = np.array([-3*step, -2*step, -step, 0.0, step, 2*step, 3*step])

dfn = amps(nearness_angles)
dfn

,angle,radian,left,right
0,-33.75,-0.589049,0.195090,0.980785
1,-22.50,-0.392699,0.382683,0.923880
2,-11.25,-0.196350,0.555570,0.831470
3,0.00,0.000000,0.707107,0.707107
4,11.25,0.196350,0.831470,0.555570
5,22.50,0.392699,0.923880,0.382683
6,33.75,0.589049,0.980785,0.195090


In [107]:
alt.Chart(dfn).mark_point(color="blue").encode(x="angle", y="left")\
+ alt.Chart(dfn).mark_point(color="red").encode(x="angle", y="right")

alt.LayerChart(...)

## Nearness resistor values

In [108]:
# resistor values in Kilo Ohm
res = [100., 103., 113., 141., 215., 439., 1670.]
dfn["resistors_right"] = res
dfn["resistors_left"] = res[::-1]

### Inverse of resistance?

In [109]:
# take inverses, as a guess for the amplification factor
dfn["res_inv_left"] = 100.0 / dfn["resistors_left"]

In [110]:
alt.Chart(dfn).mark_point().encode(x="left", y="res_inv_left")

alt.Chart(...)

This is a match at the center $(0.7, 0.7)$, but a nonlinear map, otherwise...

### Left / right ratio?

In [111]:
dfn["amp_ratio"] = dfn["left"] / dfn["right"]
dfn["resistor_ratio"] = dfn["resistors_right"] / dfn["resistors_left"]

In [112]:
alt.Chart(dfn).mark_point().encode(x="resistor_ratio", y="amp_ratio")

alt.Chart(...)

Still no linear mapping...

In [116]:
np.degrees(np.atan(dfn["amp_ratio"])) * 2

0     22.5
1     45.0
2     67.5
3     90.0
4    112.5
5    135.0
6    157.5
Name: amp_ratio, dtype: float64

In [125]:
res_ratio_degs = (np.degrees(np.atan(dfn["resistor_ratio"])) * 2).values
res_ratio_degs

array([  6.85358631,  26.40828329,  55.45116085,  90.        ,
       124.54883915, 153.59171671, 173.14641369])

In [128]:
res_ratio_degs[1:] - res_ratio_degs[:-1]

array([19.55469698, 29.04287755, 34.54883915, 34.54883915, 29.04287755,
       19.55469698])

Reverse-engineered angle degrees imply non-equal steps